# Düzenlileştirme

**Titanic** yolcularının hayatta kalma şansını etkileyen faktörlere dair anlayışımızı geliştirelim
- Yorumlaması kolay olan lojistik sınıflandırıcıları kullanacağız
- Bunu daha önce "Karar Bilimi - Lojistik Regresyon" dersinde statsmodels ile yapmıştık
- Hangi özelliklerin alakasız olduğunu / genelleştirilemediğini tespit etmek için `p-değerleri` ve istatistiksel varsayımlar kullanıyorduk
- Bu sefer, eksik/aşırı öğrenme kriterlerine dayalı olarak alakalı/alakasız özellikleri tespit etmek için `düzenlileştirme` kullanacağız
- **Amacımız `L1` ve `L2` cezalarını karşılaştırmak**

## 1. Veriyi sizin için yüklüyor ve ön işleme tabi tutuyoruz

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression




In [38]:
data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [39]:
# We build X and y

y = data["survived"]
X = data.drop(columns=["survived"])
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [40]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 2. Düzenlileştirme olmadan Lojistik Regresyon

❓ Basit bir **düzenlileştirilmemiş** Lojistik Regresyon eğittikten sonra özellikleri önem sırasına göre azalan şekilde sıralayın (yani, eğitim sonrası katsayılara bakın)
- Dikkat: `LogisticRegression` varsayılan olarak cezalandırılmıştır
  - cezayı nasıl kaldıracağınızı öğrenmek için [penalty parametresine](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) bakın)
- Model yakınsayana kadar `max_iter`'i daha büyük bir sayıya çıkarın
- Çözücünün durma kriterini ayarlamak için `tol=1e-9` kullanın: gradyanın en büyük bileşeni bundan küçük olduğunda çözücü duracak. Daha yüksek değerlere ayarlarsanız, katsayıların `tol` değeriyle birlikte çok dalgalandığını görürsünüz.

<details>
    <summary>İpucu</summary>
    <img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/05-ML/05-Model-Tuning/model_selection.png" alt="penalizing a regression" width="500">
</details>

In [41]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

log_reg = LogisticRegression(
    penalty=None,
    max_iter=10000,
    tol=1e-9,
    solver="lbfgs"
)

log_reg.fit(X_scaled, y)

coefficients = log_reg.coef_[0]

feature_importance = pd.DataFrame({
    "feature": X_scaled.columns,
    "coefficient": coefficients,
    "abs_coefficient": np.abs(coefficients)
})

feature_importance_sorted = feature_importance.sort_values(
    by="abs_coefficient",
    ascending=False
)

feature_importance_sorted



,feature,coefficient,abs_coefficient
10,embark_town_Queenstown,-21.346844,21.346844
11,embark_town_Southampton,-20.951437,20.951437
9,embark_town_Cherbourg,-20.650067,20.650067
0,pclass,5.240985,5.240985
7,class_Third,-3.803688,3.803688
6,class_First,3.707294,3.707294
5,sex_female,2.671878,2.671878
2,sibsp,-2.476885,2.476885
1,age,-2.196127,2.196127
4,fare,1.360204,1.360204


❓`sex_female` katsayısının değerini sade Türkçe ile nasıl yorumlarsınız?

<details>
    <summary>Cevap</summary>

> "Diğer tüm şeyler eşitken (yaş, bilet sınıfı vb...),
kadın olmak hayatta kalma log-oranlarınızı 2.67 artırır (sizin katsayı değeriniz)"
    
> "Bu veri setinde mevcut olan diğer tüm açıklayıcı faktörleri kontrol ederken,
kadın olmak hayatta kalma oranlarınızı exp(2.67) = 14 kat artırır"

</details>

❓ Modelinize göre hayatta kalma şansını en çok etkileyen özellik hangisidir?  
Aşağıdaki `top_1_feature` listesini bu özelliğin adıyla doldurun

In [42]:
top_1_feature = ["embark_town_Queenstown"]
top_1_feature

['embark_town_Queenstown']

In [43]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature=top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S16D5-S-regularization/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 PASSED              [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master



## 3. L2 cezalı Lojistik Regresyon

Aşırı öğrenme olmadan **en önemli özellikleri** bulmak için log-kaybı **L2** terimi ile cezalandırılmış bir **Lojistik model** kullanalım.  
Bu, "Ridge" regresörünün "sınıflandırma" karşılığıdır

❓ **Güçlü düzenlileştirilmiş** bir `LogisticRegression` oluşturun ve özelliklerini önem sırasına göre sıralayın (katsayılara bakın)
- "Güçlü düzenlileştirilmiş" ile "Sklearn'in varsayılan düzenlileştirme faktöründen daha fazla" demek istiyoruz. 
- Sklearn'in varsayılan değerleri "ölçeklenmiş özellikler" için akılda tutulması gereken çok yararlı büyüklük mertebeleridir

In [57]:
log_reg_l2 = LogisticRegression(
    penalty="l2",     
    C=0.01,           
    max_iter=10000,
    tol=1e-9,
    solver="lbfgs"
)

log_reg_l2.fit(X_scaled, y)
coefficients = log_reg_l2.coef_[0]

feature_importance_l2 = pd.DataFrame({
    "feature": X_scaled.columns,
    "coefficient": coefficients,
    "abs_coefficient": np.abs(coefficients)
})

feature_importance_l2_sorted = feature_importance_l2.sort_values(
    by="abs_coefficient",
    ascending=False
)

feature_importance_l2_sorted


,feature,coefficient,abs_coefficient
5,sex_female,0.614098,0.614098
7,class_Third,-0.295870,0.295870
0,pclass,-0.257011,0.257011
6,class_First,0.218152,0.218152
8,who_child,0.136157,0.136157
9,embark_town_Cherbourg,0.126089,0.126089
11,embark_town_Southampton,-0.109189,0.109189
1,age,-0.067343,0.067343
4,fare,0.051314,0.051314
3,parch,0.026701,0.026701


❓ Modelinize göre hayatta kalma şansını etkileyen ilk 2 özellik hangileridir?  
Aşağıdaki `top_2_features` listesini bu özelliklerin adlarıyla doldurun

In [58]:
top_2_features = ["sex_female", "class_Third"]
top_2_features

['sex_female', 'class_Third']

#### 🧪 Kodunuzu aşağıda test edin

In [59]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2=top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S16D5-S-regularization/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 PASSED                               [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master



## 4. L1 cezalı Lojistik Regresyon

Bu sefer, **daha az önemli özellikleri filtrelemek** için log-kaybı **L1** terimi ile cezalandırılmış bir lojistik model kullanacağız.  
Bu, **Lasso** regresörünün "sınıflandırma" karşılığıdır

❓ **Güçlü düzenlileştirilmiş** bir `LogisticRegression` oluşturun ve özelliklerini önem sırasına göre sıralayın

In [64]:
log_reg_l1 = LogisticRegression(
    penalty="l1",     
    C=0.5,            
    solver="liblinear",
    max_iter=10000,
    tol=1e-9
)


log_reg_l1.fit(X_scaled, y)
coefficients = log_reg_l1.coef_[0]

coef_df = pd.DataFrame({
    "feature": X_scaled.columns,
    "coef": log_reg_l1.coef_[0]
})

coef_df["abs_coef"] = coef_df["coef"].abs()

feature_importance_l1 = coef_df.sort_values(
    by="abs_coef",
    ascending=False
)

feature_importance_l1


,feature,coef,abs_coef
5,sex_female,2.439928,2.439928
2,sibsp,-1.377449,1.377449
1,age,-1.369304,1.369304
0,pclass,-1.232577,1.232577
8,who_child,0.959724,0.959724
7,class_Third,-0.521221,0.521221
6,class_First,0.376297,0.376297
9,embark_town_Cherbourg,0.326157,0.326157
3,parch,0.000000,0.000000
4,fare,0.000000,0.000000


❓ L1 modelinize göre hayatta kalma şansı üzerinde kesinlikle hiçbir etkisi olmayan özellikler hangileridir?  
Aşağıdaki `zero_impact_features` listesini bu özelliklerin adlarıyla doldurun; listeye eleman eklemeniz gerekebilir.

- Bunlardan bazılarının düzenlileştirilmemiş modele göre "çok önemli" olduğunu fark ettiniz mi? 
- Bundan sonra doğrusal modellerimizi her zaman düzenlileştireceğiz!

In [66]:
zero_impact_features = ["parch", "fare", "embark_town_Queenstown", "embark_town_Southampton"]
zero_impact_features

['parch', 'fare', 'embark_town_Queenstown', 'embark_town_Southampton']

#### 🧪 Kodunuzu aşağıda test edin

In [67]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S16D5-S-regularization/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_lasso.py::TestLasso::test_zero_impact PASSED                        [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master



# 5. Bir adım geri çekilmek

🤯 **Bu katsayılardan bazıları neden başlangıçta bu kadar yüksekti?**

Düzenlileştirme ile kaldırılan üç özelliği düşünelim:
- `embark_town_Cherbourg`
- `embark_town_Southampton`
- `embark_town_Queenstown`

Üç biniş şehri tabii ki ilişkilidir: ikisinden binmediyseniz, üçüncüsünden binmiş olmalısınız. Yani biliyoruz ki: 

$$embark\_town\_Cherbourg + embark\_town\_Southampton + embark\_town\_Queenstown = 1$$

Bu üç özellik **mükemmel çoklu doğrusal bağıntılıdır**!

**Düzenlileştirilmemiş modeller kullanılırken, bu genellikle sayısal kararsızlığa yol açar**, ki burada gördüğümüz tam olarak buydu. Ayrıca böyle bir durumda elde ettiğimiz **katsayılara gerçekten güvenemeyeceğimiz** anlamına gelir.

❗️ Bu üç çoklu doğrusal bağıntılı özellik, `embark_town` kategorik özelliğinin one hot encoding'inden gelir.

Düzenlileştirme sayesinde bu sorunu aştık: üç şehir için katsayıların çok büyük olmasını engelledi. **İşte bu yüzden neredeyse her zaman düzenlileştirme kullanacağız.**

🔍 **Başlangıçta ayarladığımız `tol` parametresini hatırlıyor musunuz?**

Düzenlileştirmenin ekstra bir bonusu da `tol` ayarlamanın daha az önemli hale gelmesi: `1e-2` ve `1e-9` arasında herhangi bir değere ayarlayabilirsiniz ve katsayılar neredeyse hiç değişmez! 💪

**🏁 Tebrikler! Not defterinizi commit etmeyi ve push etmeyi unutmayın**